### Building and Training the LSTM Model

We will now build a multi-input LSTM model that takes the padded text sequences and the other features as separate inputs. The model will consist of an embedding layer for the text input, followed by an LSTM layer. The other features will be fed into a separate dense layer. The outputs of these layers will be combined and passed through additional dense layers to make predictions.




In [ ]:
import os
import pickle

In [9]:
import os
import pickle
import numpy as np
import pandas as pd


def load_prepared_data(load_dir):
    """
    Loads the prepared data and tokenizer from specified directory in Google Drive.

    Args:
        load_dir (str): The directory path in Google Drive to load the files from.

    Returns:
        tuple: A tuple containing loaded_padded_sequences, loaded_other_features_input,
               loaded_target, loaded_df_encoded, loaded_tokenizer.
               Returns None if loading fails.
    """
    try:
        # Load numpy arrays
        padded_sequences = np.load(os.path.join(load_dir, 'padded_sequences_aligned.npy'))
        print(f"padded_sequences_aligned loaded from {os.path.join(load_dir, 'padded_sequences_aligned.npy')}")

        other_features_input = np.load(os.path.join(load_dir, 'other_features_input_aligned.npy'))
        print(f"other_features_input_aligned loaded from {os.path.join(load_dir, 'other_features_input_aligned.npy')}")

        target = np.load(os.path.join(load_dir, 'target_aligned.npy'))
        print(f"target_aligned loaded from {os.path.join(load_dir, 'target_aligned.npy')}")

        # Load DataFrame
        df_encoded = pd.read_pickle(os.path.join(load_dir, 'df_encoded_aligned.pkl'))
        print(f"df_encoded_aligned loaded from {os.path.join(load_dir, 'df_encoded_aligned.pkl')}")

        # Load tokenizer
        with open(os.path.join(load_dir, 'tokenizer.pkl'), 'rb') as handle:
            tokenizer = pickle.load(handle)
        print(f"Tokenizer loaded from {os.path.join(load_dir, 'tokenizer.pkl')}")

        print("\nAll necessary files loaded.")
        return padded_sequences, other_features_input, target, df_encoded, tokenizer

    except FileNotFoundError:
        print(f"Error loading files: One or more files not found in {load_dir}")
        return None
    except Exception as e:
        print(f"Error loading files: {e}")
        return None

# Define the directory in Google Drive where the files are saved
data_dir = '/content/drive/MyDrive/restaurant_recommendation_data/'

# Call the load_prepared_data function and assign to the same variable names
loaded_data = load_prepared_data(data_dir)

if loaded_data:
    padded_sequences, other_features_input, target, df_encoded, tokenizer = loaded_data
    print("\nData loaded successfully into original variable names.")
    # You can now use these loaded variables
    print("Shape of loaded padded sequences:", padded_sequences.shape)
    display(df_encoded.head())
else:
    print("\nFailed to load data.")

padded_sequences_aligned loaded from /content/drive/MyDrive/restaurant_recommendation_data/padded_sequences_aligned.npy
other_features_input_aligned loaded from /content/drive/MyDrive/restaurant_recommendation_data/other_features_input_aligned.npy
target_aligned loaded from /content/drive/MyDrive/restaurant_recommendation_data/target_aligned.npy
df_encoded_aligned loaded from /content/drive/MyDrive/restaurant_recommendation_data/df_encoded_aligned.pkl
Tokenizer loaded from /content/drive/MyDrive/restaurant_recommendation_data/tokenizer.pkl

All necessary files loaded.

Data loaded successfully into original variable names.
Shape of loaded padded sequences: (775, 100)


,name,rate,votes,dish_liked,approx_cost(for two people),reviews_list,processed_reviews_text,online_order_No,online_order_Yes,book_table_No,...,"cuisines_Tibetan, Momos",cuisines_Vietnamese,listed_in(type)_Buffet,listed_in(type)_Cafes,listed_in(type)_Delivery,listed_in(type)_Desserts,listed_in(type)_Dine-out,listed_in(type)_Drinks & nightlife,listed_in(city)_Banashankari,listed_in(city)_Bannerghatta Road
0,Jalsa,4.1,775,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",rated beautiful place dine inthe interior take...,False,True,False,...,False,False,True,False,False,False,False,False,True,False
1,Spice Elephant,4.1,787,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",800,"[('Rated 4.0', 'RATED\n Had been here for din...",rated dinner family turned good choose suitabl...,False,True,True,...,False,False,True,False,False,False,False,False,True,False
2,San Churro Cafe,3.8,918,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",rated ambience good enough pocket friendly caf...,False,True,True,...,False,False,True,False,False,False,False,False,True,False
3,Addhuri Udupi Bhojana,3.7,88,Masala Dosa,300,"[('Rated 4.0', ""RATED\n Great food and proper...",rated great food proper karnataka style full m...,True,False,True,...,False,False,True,False,False,False,False,False,True,False
4,Grand Village,3.8,166,"Panipuri, Gol Gappe",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",rated good restaurant neighbourhood buffet sys...,True,False,True,...,False,False,True,False,False,False,False,False,True,False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences # Import pad_sequences

# --- Define necessary variables ---
# You might need to adjust these based on your data preprocessing
max_sequence_length = 100  # Placeholder value, adjust based on your text data
vocab_size = len(tokenizer.word_index) + 1 # Assuming tokenizer is available from previous steps

# --- Hyperparameters to Tune ---
lstm_units = 64
dense_units = 64
epochs = 10
batch_size = 32
learning_rate = 0.001 # Example learning rate

# Use df_encoded as the base for aligned data and target
df_features_aligned = df_encoded.copy().reset_index(drop=True)
df_encoded_aligned = df_encoded.copy().reset_index(drop=True)

# Assuming 'rate' is the target variable for recommendation (e.g., predicting rating)
# Use the cleaned and numeric 'rate' column from df_features_aligned
# Ensure the 'rate' column exists and is numeric
if 'rate' not in df_features_aligned.columns or not pd.api.types.is_numeric_dtype(df_features_aligned['rate']):
    raise ValueError("The 'rate' column is not in df_features_aligned or is not numeric.")

# Align the target with the indices of df_features_aligned
target_aligned = df_features_aligned['rate'].values

# Re-create padded_sequences from the aligned df_features_aligned to ensure index alignment
# Assuming max_sequence_length and vocab_size are defined in a previous cell
if 'processed_reviews_text' not in df_features_aligned.columns:
     raise ValueError("The 'processed_reviews_text' column is not in df_features_aligned.")

# Debugging: Print columns of df_features_aligned before creating sequences
print("Debugging: Columns of df_features_aligned before tokenization:", df_features_aligned.columns.tolist())

try:
    sequences = tokenizer.texts_to_sequences(df_features_aligned['processed_reviews_text'].astype(str)) # Ensure text is string type
    padded_sequences_aligned = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
except KeyError as e:
    print(f"KeyError during tokenization: {e}")
    print("Debugging: Data type of 'processed_reviews_text' column:", df_features_aligned['processed_reviews_text'].dtype)
    print("Debugging: Head of 'processed_reviews_text' column:\n", df_features_aligned['processed_reviews_text'].head())
    print("Debugging: Number of NaN values in 'processed_reviews_text' column:", df_features_aligned['processed_reviews_text'].isnull().sum())
    raise # Re-raise the exception after printing debug info
except Exception as e:
    print(f"An unexpected error occurred during tokenization: {e}")
    print("Debugging: Data type of 'processed_reviews_text' column:", df_features_aligned['processed_reviews_text'].dtype)
    print("Debugging: Head of 'processed_reviews_text' column:\n", df_features_aligned['processed_reviews_text'].head())
    print("Debugging: Number of NaN values in 'processed_reviews_text' column:", df_features_aligned['processed_reviews_text'].isnull().sum())
    raise # Re-raise the exception after printing debug info


# Convert df_encoded_aligned to a numpy array for splitting, excluding the target, text columns, and other non-numeric columns
columns_to_exclude_from_other_features = ['rate', 'processed_reviews_text', 'name', 'dish_liked', 'processed_reviews', 'tokenized_reviews']
columns_for_other_features = df_encoded_aligned.columns.drop(columns_to_exclude_from_other_features, errors='ignore')

# Explicitly convert selected columns to numeric and handle NaNs
df_other_features_numeric = df_encoded_aligned[columns_for_other_features].copy()
for col in df_other_features_numeric.columns:
    df_other_features_numeric[col] = pd.to_numeric(df_other_features_numeric[col], errors='coerce')

# Fill any remaining NaNs with a suitable value (e.g., 0 or the mean)
df_other_features_numeric.fillna(0, inplace=True) # Filling with 0 as an example

# Debugging: Display dtypes and head of the numeric features dataframe
print("Debugging: df_other_features_numeric dtypes after conversion and fillna:", df_other_features_numeric.dtypes)
print("Debugging: df_other_features_numeric head after conversion and fillna:\n", df_other_features_numeric.head())

# Explicitly convert to a numeric numpy array
other_features_input_aligned = df_other_features_numeric.values.astype(np.float32)

# Add a check to confirm the dtype of the other_features_input_aligned
if not np.issubdtype(other_features_input_aligned.dtype, np.number):
    print("Debugging: other_features_input_aligned dtype after explicit conversion and fillna:", other_features_input_aligned.dtype)
    raise TypeError("other_features_input_aligned still contains non-numeric types after explicit conversion and fillna.")


# Split data into training and testing sets
# Splitting aligned data
X_text_train, X_text_test, X_other_train, X_other_test, y_train, y_test = train_test_split(
    padded_sequences_aligned, other_features_input_aligned, target_aligned, test_size=0.2, random_state=42
)

print("Shape of X_text_train:", X_text_train.shape)
print("Shape of X_other_train:", X_other_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_text_test:", X_text_test.shape)
print("Shape of X_other_test:", X_other_test.shape)
print("Shape of y_test:", y_test.shape)


# Define the model architecture

# Text Input Branch
text_input_layer = Input(shape=(max_sequence_length,), name='text_input')
embedding_layer = Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length)(text_input_layer)
lstm_layer = LSTM(lstm_units)(embedding_layer)

# Other Features Input Branch
other_features_input_layer = Input(shape=(other_features_input_aligned.shape[1],), name='other_features_input')
other_dense_layer = Dense(dense_units, activation='relu')(other_features_input_layer)

# Combine Branches
combined_features = concatenate([lstm_layer, other_dense_layer])

# Dense Layers for Prediction
dense_layer_1 = Dense(dense_units, activation='relu')(combined_features)
output_layer = Dense(1, activation='linear')(dense_layer_1) # Using linear activation for regression (predicting rating)

# Create the model
model = Model(inputs=[text_input_layer, other_features_input_layer], outputs=output_layer)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # Using Mean Squared Error and Mean Absolute Error for regression

# Model Summary
model.summary()

# Train the model
# You might need to adjust epochs, batch_size, and add validation data
history = model.fit(
    {'text_input': X_text_train, 'other_features_input': X_other_train},
    y_train,
    epochs=epochs, # Example number of epochs
    batch_size=batch_size, # Example batch size
    validation_split=0.2 # Using a validation split
)

# Evaluate the model (optional, can be done in the next step)
# loss, mae = model.evaluate({'text_input': X_text_test, 'other_features_input': X_other_test}, y_test)
# print(f"Test Loss: {loss}, Test MAE: {mae}")

Debugging: Columns of df_features_aligned before tokenization: ['name', 'rate', 'votes', 'dish_liked', 'approx_cost(for two people)', 'reviews_list', 'processed_reviews_text', 'online_order_No', 'online_order_Yes', 'book_table_No', 'book_table_Yes', 'location_BTM', 'location_Banashankari', 'location_Bannerghatta Road', 'location_Basavanagudi', 'location_City Market', 'location_JP Nagar', 'location_Jayanagar', 'location_Kumaraswamy Layout', 'location_Mysore Road', 'location_Rajarajeshwari Nagar', 'location_South Bangalore', 'location_Uttarahalli', 'location_Vijay Nagar', 'rest_type_Bakery', 'rest_type_Bakery, Dessert Parlor', 'rest_type_Bakery, Quick Bites', 'rest_type_Bar', 'rest_type_Bar, Casual Dining', 'rest_type_Beverage Shop', 'rest_type_Beverage Shop, Dessert Parlor', 'rest_type_Beverage Shop, Quick Bites', 'rest_type_Cafe', 'rest_type_Cafe, Bakery', 'rest_type_Cafe, Casual Dining', 'rest_type_Cafe, Quick Bites', 'rest_type_Casual Dining', 'rest_type_Casual Dining, Bar', 'rest_ty

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 100, 128)  │  1,455,232 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ other_features_inp… │ (None, 348)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 64)        │     49,408 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │     22,336 │ other_features_i… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 128)       │          0 │ lstm_4[0][0],     │
│ (Concatenate)       │                   │            │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 64)        │      8,256 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1)         │         65 │ dense_13[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,535,297 (5.86 MB)

 Trainable params: 1,535,297 (5.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - loss: 420.7039 - mae: 11.3490 - val_loss: 56.9963 - val_mae: 4.6608
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 30.7424 - mae: 4.2839 - val_loss: 11.9870 - val_mae: 3.3275
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 8.2486 - mae: 2.4099 - val_loss: 4.1363 - val_mae: 1.6180
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 4.2299 - mae: 1.6641 - val_loss: 2.5701 - val_mae: 1.3948
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - loss: 2.7089 - mae: 1.3832 - val_loss: 2.7104 - val_mae: 1.3963
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - loss: 2.2772 - mae: 1.2185 - val_loss: 1.0913 - val_mae: 0.8776
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - loss: 0.8271 - mae: 0.7243 - val_loss: 0.2905 - val_mae: 0.3761
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - loss: 0.2970 - mae: 0.4017 - val_loss: 0.1873 - val_mae: 0.3213
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - lo

### Evaluate the Model

Now that the model has been trained, we can evaluate its performance on the test dataset (`X_text_test`, `X_other_test`, `y_test`). We will use the `evaluate` method of the model, which will return the loss and any metrics specified during compilation (in this case, Mean Absolute Error).

In [18]:
# Evaluate the model on the test data
loss, mae = model.evaluate(
    {'text_input': X_text_test, 'other_features_input': X_other_test},
    y_test
)

print(f"Test Loss (MSE): {loss:.4f}")
print(f"Test MAE: {mae:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2077 - mae: 0.2875
Test Loss (MSE): 0.2145
Test MAE: 0.3047


## Summary of LSTM Model for Restaurant Recommendation:

### Process Followed:

1.  **Data Loading and Cleaning**: The restaurant dataset was loaded. The 'rate' column was cleaned and converted to a numeric format, and rows with missing ratings were removed. Unnecessary columns were dropped.
2.  **Text Preprocessing**: The 'reviews\_list' column was preprocessed by cleaning the text, removing stopwords, and tokenizing. A 'processed\_reviews\_text' column was created.
3.  **Data Preparation for LSTM**: The processed review text was converted into sequences of integers using a tokenizer and padded to a fixed length (`padded_sequences`). Categorical features were one-hot encoded (`df\_encoded`). The padded text sequences and other features were aligned and prepared as separate inputs for a multi-input LSTM model.
4.  **Model Building and Training**: A multi-input LSTM model was defined, compiled with 'adam' optimizer and 'mse' loss, and trained on the prepared training data.
5.  **Model Evaluation**: The trained model was evaluated on the unseen test data using MSE and MAE metrics.

### Model Performance:

*   **Test Loss (MSE)**: [Insert MSE value from output]
*   **Test MAE**: [Insert MAE value from output]

*Note: A lower MSE and MAE indicate better model performance in predicting ratings.*

### Potential Next Steps and Refinements:

1.  **Hyperparameter Tuning**: Experiment with different LSTM layers, dense layer sizes, dropout rates, optimizers, learning rates, epochs, and batch sizes to improve model performance.
2.  **Text Feature Engineering**: Explore other text vectorization techniques (e.g., GloVe, FastText embeddings) or more complex text processing.
3.  **Feature Engineering for Other Columns**: Investigate ways to incorporate information from the 'menu\_item' column if relevant for recommendations.
4.  **Different Model Architectures**: Explore other neural network architectures (e.g., GRU, attention mechanisms) or traditional machine learning models.
5.  **Recommendation Logic**: Based on the model's predictions (e.g., predicted rating), develop a recommendation logic to suggest restaurants to users. This could involve recommending restaurants with the highest predicted ratings for a user or finding similar restaurants based on the model's learned representations.
6.  **User and Item Embeddings**: For a full-fledged recommendation system, incorporating user and item embeddings trained on user-item interactions would be crucial. This model currently focuses on predicting a rating based on restaurant features.
7.  **Cross-validation**: Use cross-validation during training to get a more robust estimate of model performance.

### Finding Similar Restaurants

To recommend similar restaurants, we can use the trained LSTM model to get a combined feature representation for each restaurant. We can then calculate the similarity between the feature vector of a given restaurant and all other restaurants in the dataset. Restaurants with higher similarity scores will be considered more similar.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_restaurants(restaurant_name, df_encoded, padded_sequences, model, n_top=10):
    """
    Finds the top-N similar restaurants based on the trained model's features.

    Args:
        restaurant_name (str): The name of the restaurant to find similar ones for.
        df_encoded (pd.DataFrame): The dataframe with one-hot encoded features.
        padded_sequences (np.ndarray): The padded text sequences.
        model (tf.keras.Model): The trained multi-input LSTM model.
        n_top (int): The number of top similar restaurants to return.

    Returns:
        pd.DataFrame: A dataframe containing the top-N similar restaurants and their similarity scores.
    """
    # Ensure df_encoded and padded_sequences are aligned
    # Resetting index to ensure alignment as done before training
    df_encoded_aligned = df_encoded.reset_index(drop=True)
    # Assuming df_features_aligned is available from previous steps and aligned with df_encoded
    # If not, you might need to recreate it here based on your data loading/processing
    global df_features_aligned # Access the global variable if it's defined elsewhere
    if 'df_features_aligned' not in globals():
         print("Warning: df_features_aligned not found. Attempting to create from df_encoded.")
         df_features_aligned = df_encoded.copy().reset_index(drop=True)

    padded_sequences_aligned = pad_sequences(tokenizer.texts_to_sequences(df_features_aligned['processed_reviews_text'].astype(str)), maxlen=max_sequence_length, padding='post', truncating='post') # Recreate padded sequences from aligned df_features

    # Get the combined feature layer output from the model
    # We need to create a new model that outputs the combined features
    # Use the correct layer name 'concatenate_4' based on the model summary
    feature_layer_model = Model(inputs=model.input, outputs=model.get_layer('concatenate_4').output)

    # Prepare the input data for the feature layer model
    # Ensure other_features_input_aligned is created the same way as for training
    columns_to_exclude_from_other_features = ['rate', 'processed_reviews_text', 'name', 'dish_liked', 'processed_reviews', 'tokenized_reviews']
    columns_for_other_features = df_encoded_aligned.columns.drop(columns_to_exclude_from_other_features, errors='ignore')

    df_other_features_numeric = df_encoded_aligned[columns_for_other_features].copy()
    for col in df_other_features_numeric.columns:
        df_other_features_numeric[col] = pd.to_numeric(df_other_features_numeric[col], errors='coerce')
    df_other_features_numeric.fillna(0, inplace=True)
    other_features_input_aligned = df_other_features_numeric.values.astype(np.float32)


    # Get the feature representations for all restaurants
    all_restaurant_features = feature_layer_model.predict({'text_input': padded_sequences_aligned, 'other_features_input': other_features_input_aligned})

    # Find the index of the input restaurant
    if restaurant_name not in df_features_aligned['name'].values:
        print(f"Restaurant '{restaurant_name}' not found in the dataset.")
        return pd.DataFrame()

    restaurant_index = df_features_aligned[df_features_aligned['name'] == restaurant_name].index[0]

    # Get the feature vector for the input restaurant
    input_restaurant_features = all_restaurant_features[restaurant_index].reshape(1, -1)

    # Calculate cosine similarity between the input restaurant and all other restaurants
    similarity_scores = cosine_similarity(input_restaurant_features, all_restaurant_features)

    # Get the similarity scores for the input restaurant
    similarity_scores = list(enumerate(similarity_scores[0]))

    # Sort restaurants based on similarity scores in descending order
    sorted_similar_restaurants = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the top-N similar restaurants (excluding the input restaurant itself)
    top_similar_restaurants = []
    for i in sorted_similar_restaurants:
        if i[0] != restaurant_index:
            top_similar_restaurants.append(i)
        if len(top_similar_restaurants) >= n_top:
            break

    # Get the names and similarity scores of the top similar restaurants
    top_restaurant_indices = [i[0] for i in top_similar_restaurants]
    top_restaurant_names = df_features_aligned.loc[top_restaurant_indices, 'name'].values
    top_restaurant_scores = [i[1] for i in top_similar_restaurants]

    # Create a dataframe of similar restaurants
    similar_restaurants_df = pd.DataFrame({
        'Restaurant Name': top_restaurant_names,
        'Similarity Score': top_restaurant_scores
    })

    return similar_restaurants_df

# Example usage (you'll need to replace 'Your Restaurant Name' with an actual name from the dataset)
similar_restaurants = find_similar_restaurants('Jalsa', df_encoded, padded_sequences, model, n_top=5)
display(similar_restaurants)

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


,Restaurant Name,Similarity Score
0,Jalsa,1.000000
1,Jalsa,0.999999
2,Jalpaan,0.999995
3,Spice Elephant,0.999988
4,Spice Elephant,0.999988


### Saving the Model and Parameters

We will save the trained TensorFlow model and the tokenizer to Google Drive so they can be loaded and used in another notebook.

In [ ]:
import os
import pickle
import tensorflow as tf

def save_model_and_params(model, tokenizer, params, data_paths, save_dir):
    """
    Saves the trained model, tokenizer, model parameters, and data paths.

    Args:
        model (tf.keras.Model): The trained TensorFlow model.
        tokenizer: The fitted tokenizer.
        params (dict): Dictionary of model parameters (e.g., max_sequence_length, vocab_size).
        data_paths (dict): Dictionary of paths to data files.
        save_dir (str): The directory path in Google Drive to save the files to.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Save the trained model
    model_save_path = os.path.join(save_dir, 'lstm_recommendation_model.keras')
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Save the tokenizer
    tokenizer_save_path = os.path.join(save_dir, 'tokenizer.pkl')
    with open(tokenizer_save_path, 'wb') as handle:
        pickle.dump(tokenizer, handle)
    print(f"Tokenizer saved to {tokenizer_save_path}")

    # Save the model parameters
    params_save_path = os.path.join(save_dir, 'model_params.pkl')
    with open(params_save_path, 'wb') as handle:
        pickle.dump(params, handle)
    print(f"Model parameters saved to {params_save_path}")

    # Save data paths
    data_paths_save_path = os.path.join(save_dir, 'data_paths.pkl')
    with open(data_paths_save_path, 'wb') as handle:
        pickle.dump(data_paths, handle)
    print(f"Data paths saved to {data_paths_save_path}")

# Example usage (assuming model, tokenizer, max_sequence_length, vocab_size, data_dir are defined)
# save_dir = '/content/drive/MyDrive/restaurant_recommendation_model/'
# params = {'max_sequence_length': max_sequence_length, 'vocab_size': vocab_size}
# data_paths = {
#     'df_encoded_path': os.path.join(data_dir, 'df_encoded_aligned.pkl'),
#     'padded_sequences_path': os.path.join(data_dir, 'padded_sequences_aligned.npy'),
#     'other_features_path': os.path.join(data_dir, 'other_features_input_aligned.npy'),
#     'target_path': os.path.join(data_dir, 'target_aligned.npy')
# }
# save_model_and_params(model, tokenizer, params, data_paths, save_dir)

### Loading the Model and Parameters in Another Notebook

To load the saved model, tokenizer, and parameters in a different Colab notebook, you'll first need to mount your Google Drive and then load the files from the directory where you saved them.

In [22]:
# Execute the cell to define the save_model_and_params function
import os
import pickle
import tensorflow as tf

def save_model_and_params(model, tokenizer, params, data_paths, save_dir):
    """
    Saves the trained model, tokenizer, model parameters, and data paths.

    Args:
        model (tf.keras.Model): The trained TensorFlow model.
        tokenizer: The fitted tokenizer.
        params (dict): Dictionary of model parameters (e.g., max_sequence_length, vocab_size).
        data_paths (dict): Dictionary of paths to data files.
        save_dir (str): The directory path in Google Drive to save the files to.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Save the trained model
    model_save_path = os.path.join(save_dir, 'lstm_recommendation_model.keras')
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Save the tokenizer
    tokenizer_save_path = os.path.join(save_dir, 'tokenizer.pkl')
    with open(tokenizer_save_path, 'wb') as handle:
        pickle.dump(tokenizer, handle)
    print(f"Tokenizer saved to {tokenizer_save_path}")

    # Save the model parameters
    params_save_path = os.path.join(save_dir, 'model_params.pkl')
    with open(params_save_path, 'wb') as handle:
        pickle.dump(params, handle)
    print(f"Model parameters saved to {params_save_path}")

    # Save data paths
    data_paths_save_path = os.path.join(save_dir, 'data_paths.pkl')
    with open(data_paths_save_path, 'wb') as handle:
        pickle.dump(data_paths, handle)
    print(f"Data paths saved to {data_paths_save_path}")

# Example usage (assuming model, tokenizer, max_sequence_length, vocab_size, data_dir are defined)
# save_dir = '/content/drive/MyDrive/restaurant_recommendation_model/'
# params = {'max_sequence_length': max_sequence_length, 'vocab_size': vocab_size}
# data_paths = {
#     'df_encoded_path': os.path.join(data_dir, 'df_encoded_aligned.pkl'),
#     'padded_sequences_path': os.path.join(data_dir, 'padded_sequences_aligned.npy'),
#     'other_features_path': os.path.join(data_dir, 'other_features_input_aligned.npy'),
#     'target_path': os.path.join(data_dir, 'target_aligned.npy')
# }
# save_model_and_params(model, tokenizer, params, data_paths, save_dir)